# A Generative Adversarial Network (GAN)
A Generative Adversarial Network (GAN) consists of two neural networks: **generator** and **discriminator**, which are trained together in a competitive setting.

## Importing Packages

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt

## Data preperation
### Load and preprocess the MNIST dataset

In [6]:
(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 255.0
X_train = np.expand_dims(X_train, axis =-1) # Add channel dimension
buffer_size = 60000
batch_size = 128

### Prepare data pipeline

In [16]:
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(buffer_size).batch(batch_size)

## Generator & Discriminator

### Define the Generator

In [30]:
def build_generator(latent_dim):
    model = tf.keras.Sequential([
        layers.Dense(7*7*256, input_dim =latent_dim),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((7,7,256)),
        layers.Conv2DTranspose(128,(5,5), strides=(1,1),padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64,(5,5), strides=(2,2),padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(1,(5,5), strides=(2,2),padding='same',activation ='tanh'),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
    ]) 
    return model

### Define the Discriminator

In [26]:
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64,(5,5), strides=(2,2),padding ='same' , input_shape=(28,28,1)),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128,(5,5),strides=(2,2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1,activation='sigmoid')
    ])
    return model

### Creating generator and discriminator

In [31]:
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()

### Compile discriminator

In [32]:
discriminator.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

## Compile GAN Model

In [33]:
discriminator.trainable = False
gan_input = layers.Input(shape=(latent_dim,))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)
gan = Model(gan_input, gan_output)
gan.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy')

### Training Loop

In [40]:
def train_gan(epochs, batch_size):
    for epoch in range(epochs):
        for real_images in dataset:
            # Train discriminator
            batch_size = real_images.shape[0]
            random_latent_vectors = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_images = generator.predict(random_latent_vectors)
            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))
            d_loss_real = discriminator.train_on_batch(real_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            misleading_labels = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(random_latent_vectors, misleading_labels)

        # Print progress
        print(f"Epoch {epoch + 1}/{epochs} - D Loss: {d_loss[0]:.4f}, G Loss: {g_loss:.4f}")

        # Save and visualize generated images
        if (epoch + 1) % 10 == 0:
            plot_generated_images(epoch + 1, generator, latent_dim)

### Visualize generated images

In [43]:
# Visualize generated images
def plot_generated_images(epoch, generator, latent_dim, examples=10, dim=(1, 10)):
    noise = np.random.normal(0, 1, (examples, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # Rescale to [0, 1]

    plt.figure(figsize=(10, 1))
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.suptitle(f"Generated Images at Epoch {epoch}")
    plt.show()

# Train GAN
epochs = 3
train_gan(epochs, batch_size)

4/4 [==============================] - 0s 22ms/step


4/4 [==============================] - 0s 22ms/step


3/3 [==============================] - 0s 22ms/step
Epoch 1/3 - D Loss: 0.0465, G Loss: 0.7586
4/4 [==============================] - 0s 21ms/step


4/4 [==============================] - 0s 29ms/step


4/4 [==============================] - 0s 31ms/step


3/3 [==============================] - 0s 27ms/step
Epoch 2/3 - D Loss: 0.0569, G Loss: 0.7424
4/4 [==============================] - 0s 31ms/step


4/4 [==============================] - 0s 24ms/step


4/4 [==============================] - 0s 29ms/step


3/3 [==============================] - 0s 26ms/step
Epoch 3/3 - D Loss: 0.0453, G Loss: 0.4754
